In [61]:
import requests
from bs4 import BeautifulSoup
import random
import re


def get_urls(url, session=None):
    req = session.get(url)  # Send a GET request to the URL
    soup = BeautifulSoup(req.text, "lxml")  # Parse the response using BeautifulSoup
    return (
        link["href"] for link in soup.select("a[href^='https://dico-petitbac.com']")
    )

In [62]:
LETTER_RE = re.compile(r"^.*-[a-z]$")

In [63]:
def get_categories_letters_urls(base_url, session=None):
    for url in get_urls(
        base_url, session
    ):  # Iterate over URLs obtained from get_urls()
        for sub_url in get_urls(
            url, session
        ):  # Iterate over URLs obtained from the first iteration
            split = sub_url.split("/")
            penultimate_part = split[-2]
            if LETTER_RE.match(
                penultimate_part
            ):  # Check if penultimate part of the URL matches the pattern
                letter = penultimate_part[-1]
                category = split[-3]
                yield category, letter, sub_url

In [64]:
def get_words(args, session=None):
    category, letter, sub_url = args
    req = session.get(sub_url)  # Send a GET request to the sub-URL
    soup = BeautifulSoup(req.text, "lxml")  # Parse the response using BeautifulSoup
    return (
        category,
        letter,
        [li.text for li in soup.select("li")],
    )

In [101]:
from functools import partial
from tqdm.contrib.concurrent import thread_map

def get_dico():
    print("starting")
    base_url = "https://dico-petitbac.com/"
    dico = {}  # Initialize an empty dictionary
    with requests.Session() as session:  # Create a requests session
        categories_letters_urls = list(
            get_categories_letters_urls(base_url, session)
        )  # Get category, letter, and sub-URLs
        for category, letter, words in thread_map(
            partial(get_words, session=session), categories_letters_urls
        ):
            # Concurrently execute get_words() for each tuple of category, letter, and sub-URL
            dico.setdefault(category, {}).setdefault(letter, []).extend(
                words
            )  # Populate the dictionary with words
    print("finishing")
    return dico

/home/paul/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
import os
import json

JSON_FILEPATH = os.path.join("data", "dico.json")
with open(JSON_FILEPATH, "r") as f:
    data = json.load(f)

{'animaux': {'a': ['Abeille', 'Abeille à miel', 'Abeille charpentière', 'Abeille découpeuse', 'Abeille tueuse', 'Able de Heckel', 'Ablette', 'Abyssin', 'Acarien', 'Acarien rouge', 'Accenteur alpin', 'Accenteur mouchet', 'Achigan à grande bouche', 'Acouchi', 'Addax', 'Addax\xa0', 'Agame Barbu', 'Agame des colons', 'Agneau', 'Agouti', 'Agouti doré', 'Agouti ponctué', 'Agrion élégant', 'Agrion jouvencelle', 'Aigle', 'Aigle royal', 'Aigle royal\xa0', 'Aiglefin', 'Aigrette', 'Aigrette garzette', 'Aiguillat', 'Aiguillat noir', 'Ailurops', 'Akhal téké doré', 'Akita americain', 'Akita Inu', 'Albatros', 'Albatros hurleur', 'Alligator', 'Alligator américain', 'Alligator d’Amérique', 'Allocèbe', 'Alose', 'Alouate', 'Alouette', 'Alouette des champs', 'Alpaga', 'Amazone impériale', 'American curl', 'American staff', 'Amie', 'Ammophile des sables', 'Amour blanc', 'Anaconda', 'Anaconda géant', 'Anaconda vert', 'Anax empereur', 'Anchois', 'Âne', 'Ane de Provence', 'Ane grand noir du Berry', 'Ane sauva

In [99]:
def find_word(lettre):
    for categories in data:
        value = random.choice(list(data[categories][lettre]))
        print(categories, ":", value)


find_word("b")

animaux : Barge à queue noire
pays : Bangladesh


IndexError: Cannot choose from an empty sequence